In [1]:
import os
# print number of images in each directory
for directory in os.listdir('Data'):
    if os.path.isdir(f'Data/{directory}'):
        print(f'{directory}: {len(os.listdir(f"Data/{directory}"))}')

AK_images: 1202
ance_images: 295
BCC_images: 3323
BKL_images: 2624
DF_images: 239
eczema_images: 322
MEL_images: 4918
NV_images: 12875
psoriasis_images: 420
SCC_images: 628
VASC_images: 253


In [12]:
#python version 3.11
%pip install tensorflow
%pip install keras
%pip install matplotlib
%pip install numpy
%pip install pandas
%pip install scikit-learn
%pip install PIL


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement PIL (from versions: none)
ERROR: No matching distribution found for PIL


In [8]:
import random
import shutil

def sample_images(directory):
    shutil.rmtree(f'Data/sample/{directory}', ignore_errors=True)
    images = os.listdir(directory)
    n = 1000
    if len(images) < 1000:
        n=len(images)
    sample = random.sample(images, n)
    os.makedirs(f'Data/sample/{directory}', exist_ok=True)
    for image in sample:
        shutil.copyfile(f'{directory}/{image}', f'Data/sample/{directory}/{image}')

for directory in os.listdir('Data'):
    if os.path.isdir(f'Data/{directory}') and directory != 'sample':
        sample_images(f'Data/{directory}')
        print(f'{directory}: {len(os.listdir(f"Data/sample/Data/{directory}"))}')



AK_images: 1000
ance_images: 295
BCC_images: 1000
BKL_images: 1000
DF_images: 239
eczema_images: 322
MEL_images: 1000
NV_images: 1000
psoriasis_images: 420
SCC_images: 628
VASC_images: 253


In [13]:
from keras.preprocessing.image import ImageDataGenerator
import os
from PIL import Image

# Path to the directory containing image folders
data_dir = "Data/sample/Data"

# Desired number of images after augmentation
target_count = 1500

# Parameters for data augmentation
data_generator = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    validation_split=0.2
)
batch_size = 16
# Iterate through each folder
# Create a generator for images in this folder
generator = data_generator.flow_from_directory(
    data_dir,
    target_size=(300, 300),  
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'  # Use training subset for augmentation
)
for folder in os.listdir(data_dir):
    folder_path = os.path.join(data_dir, folder)
    if os.path.isdir(folder_path):
        print(f"Augmenting images in folder: {folder}")
        
        # Augment images until the target count is reached
        batches = 0
        for x_batch, y_batch in generator:
            for i in range(len(x_batch)):
                # Save augmented images
                img = x_batch[i]
                # Save augmented image using appropriate filename
                pil_img = Image.fromarray((x_batch[i] * 255).astype(np.uint8))  # Convert normalized pixels back to uint8
                # Save augmented image using appropriate filename
                pil_img.save(os.path.join(folder_path, f"augmented_image_{batches * batch_size + i}.jpg"))
            batches += 1
            if batches * batch_size >= target_count:
                # Target count reached, break the loop
                break

print("Data augmentation completed.")


Found 5728 images belonging to 11 classes.
Augmenting images in folder: AK_images
Augmenting images in folder: ance_images
Augmenting images in folder: BCC_images
Augmenting images in folder: BKL_images
Augmenting images in folder: DF_images
Augmenting images in folder: eczema_images
Augmenting images in folder: MEL_images
Augmenting images in folder: NV_images
Augmenting images in folder: psoriasis_images
Augmenting images in folder: SCC_images
Augmenting images in folder: VASC_images
Data augmentation completed.


In [18]:
import os
# print number of images in each directory
for directory in os.listdir('Data/sample/Data'):
    if os.path.isdir(f'Data/sample/Data/{directory}'):
        print(f'{directory}: {len(os.listdir(f"Data/sample/Data/{directory}"))}')

AK_images: 2504
ance_images: 1799
BCC_images: 2504
BKL_images: 2504
DF_images: 1743
eczema_images: 1826
MEL_images: 2504
NV_images: 2504
psoriasis_images: 1924
SCC_images: 2132
VASC_images: 1757


In [20]:
import random
import os
def reduce_images(directory):
    images = os.listdir(directory)
    n = 1500
    if len(images) > n:
        sample = random.sample(images, n)
        for image in images:
            if image not in sample:
                os.remove(f'{directory}/{image}')
for directory in os.listdir('Data/sample/Data'):
    if os.path.isdir(f'Data/sample/Data/{directory}'):
        reduce_images(f'Data/sample/Data/{directory}')
        print(f'{directory}: {len(os.listdir(f"Data/sample/Data/{directory}"))}')

AK_images: 1500
ance_images: 1500
BCC_images: 1500
BKL_images: 1500
DF_images: 1500
eczema_images: 1500
MEL_images: 1500
NV_images: 1500
psoriasis_images: 1500
SCC_images: 1500
VASC_images: 1500


In [1]:
#train test split 
import os
import shutil
import random
# Create a new directory
os.makedirs('Data/sample/train', exist_ok=True)
os.makedirs('Data/sample/test', exist_ok=True)
# Iterate through each folder
for folder in os.listdir('Data/sample/Data'):
    folder_path = os.path.join('Data/sample/Data', folder)
    if os.path.isdir(folder_path):
        # Get a random sample of images
        images = os.listdir(folder_path)
        random.shuffle(images)
        split_index = int(0.8 * len(images))
        train_images = images[:split_index]
        test_images = images[split_index:]
        # Move images to train/test folders
        for image in train_images:
            source = os.path.join(folder_path, image)
            target = os.path.join('Data/sample/train', folder, image)
            os.makedirs(os.path.dirname(target), exist_ok=True)
            shutil.copyfile(source, target)
        for image in test_images:
            source = os.path.join(folder_path, image)
            target = os.path.join('Data/sample/test', folder, image)
            os.makedirs(os.path.dirname(target), exist_ok=True)
            shutil.copyfile(source, target)



